Testing influx. 
https://docs.influxdata.com/influxdb/cloud/api-guide/client-libraries/python/#:~:text=Write%20data%20to%20InfluxDB%20with%20Python&text=In%20your%20Python%20program%2C%20import,to%20write%20data%20to%20InfluxDB.&text=Define%20a%20few%20variables%20with,bucket%2C%20organization%2C%20and%20token.&text=Instantiate%20the%20client.,url%20%2C%20org%20%2C%20and%20token%20.

In [5]:
import influxdb_client
import certifi # we need it to support their F**** certification problems. 
from influxdb_client.client.write_api import SYNCHRONOUS

bucket = "pg_test_1"
org = "Metis_Light"
token = "7tNrDTloBmDAX4cjAWUW34zk6LrGt_d3doSYylzNonuZslNL7txoDQzwylYBAZYxMEPKWLDvAxAvY3z_2BEmzQ=="
# Store the URL of your InfluxDB instance
# url="https://us-west-2-1.aws.cloud2.influxdata.com" # from the original demo
url="https://us-east-1-1.aws.cloud2.influxdata.com/" # The actual cloud

client = influxdb_client.InfluxDBClient(
    url=url,
    token=token,
    org=org,
    # https://stackoverflow.com/questions/69401104/influxdb-2-0-certificate-verify-failed-certificate-has-expired-ssl-c1129
    ssl_ca_cert=certifi.where()
)

# Write script
write_api = client.write_api(write_options=SYNCHRONOUS)

p = influxdb_client.Point("my_measurement").tag("location", "Prague").field("temperature", 25.3)
write_api.write(bucket=bucket, org=org, record=p)


# Query script
query_api = client.query_api()
query = 'from(bucket:"pg_test_1")\
|> range(start: -10m)\
|> filter(fn:(r) => r._measurement == "my_measurement")\
|> filter(fn:(r) => r.location == "Prague")\
|> filter(fn:(r) => r._field == "temperature")'
result = query_api.query(org=org, query=query)
results = []
for table in result:
    for record in table.records:
        results.append((record.get_field(), record.get_value()))

print(results)
# [(temperature, 25.3)]

[('temperature', 25.3), ('temperature', 25.3), ('temperature', 25.3), ('temperature', 25.3)]
